In [0]:
# Define the list (or any other type of iterable) of splits to save
saved_splits = range(0, 5) # Saves the first 5 splits

In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

import logging
import json
import re

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
def replace_empty_string(col):
    return F.when(col == "", None).otherwise(col)

def transform(_df):
    # Create the col of author IDs
    _df = _df.withColumn('Author_ID', F.col('authors._id'))
    # Delete entries where any author ID is null
    _df = _df.where("!exists(Author_ID, x -> x is null)")
    # Drop entries with 1-word titles or empty authors or nonexistant _id or any nonexistant author id.
    # Also removes empty or missing references.
    _df = (_df.filter((F.size(F.col('authors')) > 0) & # By default F.size() returns -1 if the value is null.
                      (F.size(F.split(F.col('title'), ' ')) > 1) &  
                      (F.col('_id') != '') & 
                      (F.col('_id').isNotNull()) & 
                      ~(F.array_contains(F.col('references'), '')) & 
                      ~(F.array_contains(F.col('Author_ID'), ''))))
    # Remove all null references
    _df = _df.withColumn('references', F.expr('filter(references, x -> x is not null)'))    
    # Remove entries that are forewords
    _df = _df.filter(~F.lower(F.col("title")).contains("foreword"))
    # Convert n_citation data type to int
    _df = _df.withColumn('n_citation', F.col('n_citation').cast('int'))
    # Replace empty language values with null.
    _df = _df.withColumn('lang', F.when(F.col('lang') == '', None).otherwise(F.col('lang')))
    # Replace empty 'keyword' and 'fos' arrays with null values.
    _df = (_df.withColumn('keywords', F.when(F.size(F.col('keywords')) == 0, None).otherwise(F.col('keywords')))
              .withColumn('fos', F.when(F.size(F.col('fos')) == 0, None).otherwise(F.col('fos'))))
    # Replace non-numeric page numbers with nulls and convert column type to int. Then replace 0 page numbers with nulls as well.
    _df = (_df.withColumn('page_start', F.when(F.col('page_start').cast('int').isNotNull(), F.col('page_start')).otherwise(None)) # replace non-numeric page numbers with null
              .withColumn('page_end', F.when(F.col('page_end').cast('int').isNotNull(), F.col('page_end')).otherwise(None))
              .withColumn('page_start', F.col('page_start').cast('int')) # convert column type to int
              .withColumn('page_end', F.col('page_end').cast('int'))
              .withColumn('page_start', F.when(F.col('page_start') == 0, None).otherwise(F.col('page_start'))) # replace 0 page numbers with null as well
              .withColumn('page_end', F.when(F.col('page_end') == 0, None).otherwise(F.col('page_end'))))
    # Replace empty dois with nulls.
    _df = _df.withColumn('doi', F.when(F.col('doi') == '', None).otherwise(F.col('doi')))
    # Replace empty years with nulls and change data type to int.
    _df = (_df.withColumn('year', F.when(F.col('year') == 0, None).otherwise(F.col('year')))
              .withColumn('year', F.col('year').cast('int')))
    # Replace non-numeric volume and issue numbers with null and convert data types to int. Then repalce 0 values with null as well.
    _df = (_df.withColumn('volume', F.when(F.col('volume').cast('int').isNotNull(), F.col('volume')).otherwise(None)) # replace non-numeric values
              .withColumn('issue', F.when(F.col('issue').cast('int').isNotNull(), F.col('issue')).otherwise(None))
              .withColumn('volume', F.col('volume').cast('int')) # convert column type to int
              .withColumn('issue', F.col('issue').cast('int'))
              .withColumn('volume', F.when(F.col('volume') == 0, None).otherwise(F.col('volume'))) # replace 0 issue and volume numbers with null as well.
              .withColumn('issue', F.when(F.col('issue') == 0, None).otherwise(F.col('issue'))))
    
    # Replace empty strings in some columns with nulls
    venue = F.col("venue")
    for col in ["_id", "issn", "name", "name_d", "name_s", "online_issn", "publisher", "raw", "raw_zh", "t"]:
        venue = venue.withField(col, replace_empty_string(F.col(f"venue.{col}")))  
    _df = (
        _df
        .withColumn("venue", venue)
        .withColumn("issn", replace_empty_string(F.col("issn")))
        .withColumn("isbn", replace_empty_string(F.col("isbn")))
        .withColumn("isbn", F.when(F.col("isbn") == "isbn", None).otherwise(F.col("isbn")))
        .withColumn("issn", F.when(F.col("issn") == "issn", None).otherwise(F.col("issn")))
    )
    # fix incorrect issn
    _df = (_df
               .withColumn("issn",
                           F.when(F.length(F.col("issn")) == 9, F.col("issn"))
                           .when(F.length(F.col("issn")) == 8, F.concat(F.col("issn").substr(1, 4), F.lit("-"), F.col("issn").substr(5, 4)))
                           .when(F.col("issn").contains("E-ISBN"), F.col("issn").substr(1, 9))
                           .otherwise(None)
                          )
               .withColumn("venue", 
                           F.col("venue")
                           .withField("issn", F.coalesce(F.col("venue.issn"), F.col("issn")))
                           )
               .drop("issn")
              )
    # replace venue with null fields with null
    venue_is_empty = (
        F.col("venue.issn").isNull() &
        F.col("venue.name").isNull() &
        F.col("venue.name_d").isNull() &
        F.col("venue.name_s").isNull() &
        F.col("venue.online_issn").isNull() &
        F.col("venue.publisher").isNull() &
        F.col("venue.raw").isNull() &
        F.col("venue.raw_zh").isNull()
    )
    _df = _df.withColumn("venue", F.when(venue_is_empty, None).otherwise(F.col("venue")))
    # remove rows with null venues
    _df = _df.filter(F.col("venue").isNotNull())
    # coalescing venue._id and venue.issn to make up for missing ids
    _df = _df.withColumn("venue", F.col("venue").withField("_id", F.coalesce(F.col("venue._id"), F.col("venue.issn"))))
    # removing rows with venue id null
    _df = _df.filter(F.col("venue._id").isNotNull())
    return _df

In [0]:
for n in saved_splits:
    df = spark.read.option("multiline", True).json(f'dbfs:/user/dblpv13/dblpv13.{n}.json.gz')
    
    df = transform(df)
    df.write.parquet(f'dbfs:/user/dblpv13/dblpv13.{n}.parquet')
    logger.info(f'Wrote split {n} to parquet. ({df.count()}) rows')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-2821859725189327> in <module>
      3 
      4     df = transform(df)
----> 5     df.write.parquet(f'dbfs:/user/dblpv13/dblpv13.{n}.parquet')
      6     logger.info(f'Wrote split {n} to parquet. ({df.count()}) rows')

/databricks/spark/python/pyspark/sql/readwriter.py in parquet(self, path, mode, partitionBy, compression)
    883             self.partitionBy(partitionBy)
    884         self._set_opts(compression=compression)
--> 885         self._jwrite.parquet(path)
    886 
    887     def text(self, path, compression=None, lineSep=None):

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, se

In [0]:
# Test reading a parquet file
split = 0
df_parq = spark.read.parquet(f'dbfs:/user/dblpv13/dblpv13.{split}.parquet')
logger.info(f'Processed split {split} contains {df_parq.count()} rows')
df_parq.printSchema()

INFO:__main__:Processed split 0 contains 156949 rows
root
 |-- _id: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- name_zh: string (nullable = true)
 |    |    |-- oid: string (nullable = true)
 |    |    |-- oid_zh: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- org_zh: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- orgs_zh: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- 

In [0]:
display(df_parq.limit(10))

_id,abstract,authors,doi,fos,isbn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year,Author_ID
53e99784b7602d9701f3e151,,"List(List(53f46797dabfaeb22f542630, null, null, null, Jairo Rocha, null, null, null, null, null, null, null, null, null, null), List(54328883dabfaeb4c6a8a699, null, null, null, Theo Pavlidis, null, null, null, null, null, null, null, null, null, null))",10.1109/ICDAR.1993.395663,"List(Intelligent character recognition, Pattern recognition, Computer science, Feature (computer vision), Document processing, Handwriting recognition, Optical character recognition, Feature extraction, Feature (machine learning), Artificial intelligence, Intelligent word recognition)",null,null,"List(handwriting recognition, prototypes, image segmentation, computer science, expert systems, knowledge base, pattern recognition, usability, optical character recognition, shape, feature extraction)",en,17,605,602,null,"List(53e99cf5b7602d97025ace63, 557e8a7a6fee0fe990caa63d, 53e9a96cb7602d97032c459a, 53e9b929b7602d9704515791, 557e59ebf6678c77ea222447)",A solution to the problem of touching and broken characters.,List(http://dx.doi.org/10.1109/ICDAR.1993.395663),"List(53a72a4920f7420be8bfa51b, null, null, International Conference on Document Analysis and Recognition, null, null, null, ICDAR-1, null, null, null, null, 0)",null,1993,"List(53f46797dabfaeb22f542630, 54328883dabfaeb4c6a8a699)"
53e99784b7602d9701f3e15d,"As process variations become a significant problem in deep sub-micron technology, a shift from deterministic static timing analysis to statistical static timing analysis for high-performance circuit designs could reduce the excessive conservatism that is built into current timing design methods. We address the timing yield problem for sequential circuits and propose a statistical approach to handle it. We consider the spatial and path reconvergence correlations between path delays, set-up time and hold time constraints, and clock skew due to process variations. We propose a method to get the timing yield based on the delay distributions of register-to-register paths in the circuit On average, the timing yield results obtained by our approach have average errors of less than 1.0% in comparison with Monte Carlo simulation. Experimental results show that shortest path variations and clock skew due to process variations have considerable impact on circuit timing, which could bias the timing yield results. In addition, the correlation between longest and shortest path delays is not significant.","List(List(53f43b03dabfaedce555bf2a, null, null, null, Min Pan, null, null, null, null, null, null, null, null, null, null), List(53f45ee9dabfaee43ecda842, null, null, null, Chris C. N. Chu, null, null, null, null, null, null, null, null, null, null), List(53f42e8cdabfaee1c0a4274e, null, null, null, Hai Zhou, null, null, null, null, null, null, null, null, null, null))",10.1109/ISCAS.2005.1465124,"List(Delay calculation, Timing failure, Monte Carlo method, Sequential logic, Statistical static timing analysis, Shortest path problem, Computer science, Algorithm, Clock skew, Static timing analysis, Statistics)",0-7803-8834-8,null,"List(sequential circuits, statistical distributions, set-up time constraints, register-to-register paths, statistical static timing analysis, integrated circuit modelling, parameter estimation, statistical analysis, circuit model, path delays, deep sub-micron technology, timing, delay distributions, delays, circuit timing, shortest path variations, hold time constraints, integrated circuit yield, process variations, integrated circuit layout, high-performance circuit designs, clock skew, timing yield estimation, deterministic static timing analysis, monte carlo simulation, design method, static timing analysis, design methodology, process variation, shortest path, registers, circuit design, circuit analysis)",en,28,null,2461,//static.aminer.org/pdf/PDF/000/423/329/timing_yie